In [222]:
from pyspark.sql import SparkSession

"""make an instance of a SparkSession called 'spark'."""
spark = SparkSession.builder.master('local').getOrCreate()


In [223]:
import os
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType

# custom_schema = StructType([
#     StructField('index', IntegerType()),
#     StructField('artist_popularity', LongType()),
#     StructField('followers', LongType()),
#     StructField('genres', StringType()),
#     StructField('id', StringType()),
#     StructField('name', StringType()),
#     StructField('track_id', StringType()),
#     StructField('track_name_prev', StringType()),
#     StructField('type', StringType())])
# Read the Spotify artists CSV file into a Spark DataFrame
spark_df = (spark.read.format("csv"
                        ).options(header="true"
                        ).schema("index int, artist_popularity int, followers int, genres string, id string, name string, track_id string, track_name_prev string, type string"
                        ).load("data/spotify_artists.csv"))


In [ ]:

spark_df.withColumnRenamed('', 'index').columns



In [ ]:
"""Profile the Data:"""
# Show a description (summary) of the Spark DataFrame.
spark_df.describe


In [224]:
# Print the schema of the DataFrame.
spark_df.printSchema()

root
 |-- index: integer (nullable = true)
 |-- artist_popularity: integer (nullable = true)
 |-- followers: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- track_id: string (nullable = true)
 |-- track_name_prev: string (nullable = true)
 |-- type: string (nullable = true)



In [ ]:
# Select and show just the first 10 values in the 'name' and 'genres' columns.
spark_df.select('name', 'genres').show(8)


In [225]:
from pyspark.sql.functions import regexp_replace
# Where the genre is an empty list, replace it with ['elevator music'].
spark_df = spark_df.withColumn('genres', regexp_replace(
    'genres', r"\[\]", "['elevator music']"))
spark_df.select('name', 'genres').show(8)


+--------------------+--------------------+
|                name|              genres|
+--------------------+--------------------+
|       Juliano Cezar|['sertanejo', 'se...|
|      The Grenadines|  ['elevator music']|
|             Gangway| ['danish pop rock']|
|               FADES|['uk alternative ...|
| Jean-Pierre Guignon|  ['french baroque']|
|              Filhos|  ['elevator music']|
|                Eloq|  ['elevator music']|
|              Fravær|  ['elevator music']|
|       Camille Pépin|  ['elevator music']|
|Pepe Willberg & T...|['classic finnish...|
+--------------------+--------------------+
only showing top 10 rows



In [226]:
from pyspark.sql.functions import col
"""For the columns 'artist_popularity' and 'followers', cast the data type as integers."""
# done in the schema, but ill throw in the code, so you know i know it
spark_df = spark_df.withColumn('artist_popularity', col(
                    'artist_popularity').cast(IntegerType()))
spark_df = spark_df.withColumn('followers', col(
                    'followers').cast(IntegerType()))



In [ ]:
# Sort the data in descending order by number of followers.
spark_df.select('*').sort('followers', ascending=False).show(5)


In [229]:
"""process to change the artist popularity to percent and rename the column"""
from pyspark.sql.functions import udf

# user defined function to divide x by 100
pop_contest = udf(lambda x: x/100)
# apply our udf to the column artist popularity
pop_contest_df = spark_df.withColumn('artist_popularity', 
                            pop_contest(spark_df['artist_popularity']))
# Rename the column 'popularity_percent'.
pop_contest_df = spark_df.withColumnRenamed('artist_popularity', 'popularity_percent')
# print the least cool kids to chow our function is working
pop_contest_df.select('popularity_percent', 'followers', 'name', 'genres').sort('popularity_percent').show(8)


+------------------+---------+--------------------+------------------+
|popularity_percent|followers|                name|            genres|
+------------------+---------+--------------------+------------------+
|                 0|        4|         Adah Sharma|['elevator music']|
|                 0|        0|       Kay Kay Menon|['elevator music']|
|                 0|       23|      Woody Woodbury|['elevator music']|
|                 0|        5|              Caelum|['elevator music']|
|                 0|        1|        Chemeca Gant|['elevator music']|
|                 0|        8|           Tobi Tobi|['elevator music']|
|                 0|        4|Amitabh Bachchan,...|['elevator music']|
|                 0|       19|      Shelly Winters|['elevator music']|
+------------------+---------+--------------------+------------------+
only showing top 8 rows



In [ ]:
"""Extract Information"""


# Save as Parquet

# Your /data directory shouldn't be pushed to GitHub, so this file won't show up in the submitted repository.


In [ ]:
# Show only the values in the DataFrame that have 'Queen' in the name.
spark_df.select('*').filter(spark_df.name.contains('Queen')).show(8)

In [230]:
# Group the data by artist popularity, and show the count for each group.


spark_df = spark_df.withColumn('artist_popularity', spark_df[
    'artist_popularity'].cast('double'))
spark_df.printSchema()

spark_df.groupBy('artist_popularity').sum('artist_popularity').show(8)

root
 |-- index: integer (nullable = true)
 |-- artist_popularity: double (nullable = true)
 |-- followers: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- track_id: string (nullable = true)
 |-- track_name_prev: string (nullable = true)
 |-- type: string (nullable = true)

+-----------------+----------------------+
|artist_popularity|sum(artist_popularity)|
+-----------------+----------------------+
|              8.0|                4136.0|
|             70.0|               10430.0|
|             67.0|               14204.0|
|              0.0|                   0.0|
|             69.0|               10488.0|
|              7.0|                3087.0|
|             88.0|                 616.0|
|             49.0|               46207.0|
|             98.0|                  98.0|
|             29.0|               34278.0|
+-----------------+----------------------+
only showing top 10 rows



In [232]:
# Save as Parquet
spark_df.write.parquet('data/spotify.parquet')
# Lastly, write the code to save the DataFrame as a Parquet file in the /data directory.
